<a href="https://colab.research.google.com/github/KAFE45/Colab-LabBCI/blob/main/XDFtoFIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyxdf
!pip install mne

In [24]:
import pyxdf
import mne
import io  # นำเข้า io
from google.colab import files
import numpy as np
import pandas as pd

# อัปโหลดไฟล์จากเครื่อง
uploaded = files.upload()
file_name = list(uploaded.keys())[0]  # ตรวจสอบชื่อไฟล์

# โหลดไฟล์ XDF
streams, header = pyxdf.load_xdf(io.BytesIO(uploaded[file_name]))

# แสดงชื่อสตรีมทั้งหมด
for i, stream in enumerate(streams):
    print(f"Stream {i}: {stream['info']['name']}")

# ตรวจสอบข้อมูลในสตรีม EEG
eeg_stream = [s for s in streams if 'eeg' in s['info']['name']][0]
print("EEG Stream Info:")
print(f"Name: {eeg_stream['info']['name']}")
print(f"Shape: {eeg_stream['time_series'].shape}")
print(f"Sample Rate: {eeg_stream['info']['nominal_srate']} Hz")

# ตรวจสอบข้อมูลในสตรีม Markers
marker_stream = [s for s in streams if 'PsychoPyMarkers' in s['info']['name']]
if marker_stream:
    markers = marker_stream[0]['time_series']
    marker_times = marker_stream[0]['time_stamps']
    print("Markers Stream Info:")
    print(f"Name: {marker_stream[0]['info']['name']}")
    print(f"Shape: {marker_stream[0]['time_series'].shape}")
else:
    markers = []
    marker_times = []
    print("ไม่มี markers ในข้อมูล")

# สร้างข้อมูล MNE Raw จากสตรีม EEG
data = eeg_stream['time_series'].T  # เปลี่ยนรูปร่างเพื่อให้ MNE ใช้ได้
sfreq = float(eeg_stream['info']['nominal_srate'][0])
ch_names = [ch['label'][0] for ch in eeg_stream['info']['desc'][0]['channels'][0]['channel']]

# สร้าง RawArray สำหรับ MNE
info = mne.create_info(ch_names, sfreq, ch_types='eeg')
raw = mne.io.RawArray(data, info)

# ตั้งชื่อไฟล์ FIF ที่จะบันทึก
fif_file_name = input("กรุณาใส่ชื่อไฟล์ FIF (เช่น 'output.fif'): ")
if not fif_file_name.endswith('.fif'):
    fif_file_name += '.fif'  # เพิ่มนามสกุล .fif หากไม่มี

# หากมี markers ให้เพิ่ม annotations
if len(marker_times) > 0 and len(markers) > 0:
    # สร้าง annotations สำหรับ markers
    descriptions = [marker[0] for marker in markers]  # ใช้คำอธิบายจาก markers
    annotations = mne.Annotations(onset=marker_times, duration=[0.0] * len(marker_times), description=descriptions)
    raw.set_annotations(annotations)

    # บันทึกข้อมูล Markers Stream Info ใน annotations
    markers_info = f"Markers Stream Info:\nName: {marker_stream[0]['info']['name']}\nShape: {marker_stream[0]['time_series'].shape}"
    raw.set_annotations(mne.Annotations(onset=[0], duration=[0], description=[markers_info]))

# บันทึกไฟล์ FIF ใหม่พร้อม annotations
raw.save(fif_file_name, overwrite=True)

# ดาวน์โหลดไฟล์ FIF
files.download(fif_file_name)


Saving sub-DataSSVEP551_ses-S002_task-Default_run-001_eeg.xdf to sub-DataSSVEP551_ses-S002_task-Default_run-001_eeg (7).xdf
Stream 0: ['eeg']
Stream 1: ['PsychoPyMarkers']
EEG Stream Info:
Name: ['eeg']
Shape: (38776, 24)
Sample Rate: ['256'] Hz
Markers Stream Info:
Name: ['PsychoPyMarkers']
Shape: (10, 1)
Creating RawArray with float64 data, n_channels=24, n_times=38776
    Range : 0 ... 38775 =      0.000 ...   151.465 secs
Ready.
กรุณาใส่ชื่อไฟล์ FIF (เช่น 'output.fif'): L
Writing /content/L.fif
Closing /content/L.fif
[done]


<ipython-input-24-77e2f9758b94>:58: RuntimeWarning: Omitted 10 annotation(s) that were outside data range.
  raw.set_annotations(annotations)
<ipython-input-24-77e2f9758b94>:65: RuntimeWarning: This filename (/content/L.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(fif_file_name, overwrite=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
import mne

# เส้นทางไปยังไฟล์ FIF
file_path = '/content/L.fif'

# โหลดไฟล์ FIF
raw = mne.io.read_raw_fif(file_path, preload=True)

# แสดงข้อมูลทั่วไป
print(f"ชื่อช่องสัญญาณ: {raw.ch_names}")
print(f"อัตราการสุ่ม: {raw.info['sfreq']} Hz")
print(f"จำนวนช่องสัญญาณ: {len(raw.ch_names)}")
print(f"ขนาดข้อมูล: {raw.n_times} samples")

# แสดงประเภทของข้อมูลช่องสัญญาณ
channel_types = raw.get_channel_types()
print(f"ประเภทของข้อมูล: {channel_types}")

# ตรวจสอบ markers
if raw.annotations:
    print(f"จำนวน markers: {len(raw.annotations)}")
    print("Markers:")
    for ann in raw.annotations:
        print(f" - {ann['description']} at {ann['onset']} seconds")
else:
    print("ไม่มี markers ในข้อมูล")

# เพิ่มเติม: แสดงรายละเอียดของ markers ทั้งหมดที่โหลดมาจาก PsychoPyMarkers
if len(marker_times) > 0 and len(markers) > 0:
    print("\nรายละเอียดของ Markers จาก PsychoPyMarkers:")
    for i, (timestamp, marker) in enumerate(zip(marker_times, markers)):
        print(f"Marker {i + 1}: {marker[0]} at {timestamp:.2f} seconds")
else:
    print("ไม่มี markers ในข้อมูล PsychoPyMarkers")


# แสดงตัวอย่างข้อมูลในช่องสัญญาณแรก
data, times = raw[:, :10]  # แสดงข้อมูล 10 ตัวอย่างแรก
print(f"ข้อมูล 10 ตัวอย่างแรกในช่อง {raw.ch_names[0]}: {data[0]}")


Opening raw data file /content/L.fif...
Isotrak not found
    Range : 0 ... 38775 =      0.000 ...   151.465 secs
Ready.
Reading 0 ... 38775  =      0.000 ...   151.465 secs...
ชื่อช่องสัญญาณ: ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4', 'O1', 'Channel 6', 'Channel 7', 'Channel 8', 'Fz', 'Channel 10', 'Channel 11', 'Channel 12', 'Channel 13', 'O2', 'Channel 15', 'Channel 16', 'Channel 17', 'Cz', 'Pz', 'A2', 'Channel 21', 'Channel 22', 'Channel 23', 'Channel 24']
อัตราการสุ่ม: 256.0 Hz
จำนวนช่องสัญญาณ: 24
ขนาดข้อมูล: 38776 samples
ประเภทของข้อมูล: ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
จำนวน markers: 1
Markers:
 - Markers Stream Info:
Name: ['PsychoPyMarkers']
Shape: (10, 1) at 0.0 seconds

รายละเอียดของ Markers จาก PsychoPyMarkers:
Marker 1: 13 at 13241.40 seconds
Marker 2: 2 at 13241.40 seconds
Marker 3: 12 at 13271.39 seconds
Marker 4: 3 at 13271.3

<ipython-input-26-89cff3b441da>:7: RuntimeWarning: This filename (/content/L.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True)
